### word2vec 기반 (= Word Embedding)
- 단어의 의미는 그 단어 주변 단어의 분포로 이해될 수 있음
- 단어의 의미는 단어 벡터 안에 인코딩
- 단순히 출현횟수만을 고려하는 것이 아닌 단어 위치, 순서도 고려하는 방법
    - CBOW : 주변 단어로 중심 단어를 예측하는 방법
        ex) 다음 달부터 경기도 성남시 공원에서 치킨 등을 주문하면...

        다음,  경기도  >>>  '달부터'
    - Skip-gram : 중심 단어로 주변 단어를 예측하는 방법(더 많이 사용되는 방법)
        ex) 다음 달부터 경기도 성남시 공원에서 치킨 등을 주문하면...

        성남시  >>>  '경기도', '공원에서'

- vs doc2vec 문서를 벡터로 / char2vec 문자를 벡터로


In [2]:
import glob

pos_review=(glob.glob("Data/imdb/train/pos/*.txt"))[:100]   # 긍정리뷰데이터 100개 로딩
lines_pos = []
for i in pos_review:
    try:
        f = open(i, 'r') # 파일을 읽음
        temp = f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except:
        continue
len(lines_pos)

100

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim.models.word2vec import Word2Vec
import pandas as pd

tokenizer = RegexpTokenizer('[\w]+')   # 알파벳, 숫자, _ 외에는 버린다
stop_words = stopwords.words('english')

# 단어 추출
text = []
for line in lines_pos:
    words = line.lower()
    tokens = tokenizer.tokenize(words)
    stopped_tokens = [i for i in list(set(tokens)) if i not in stop_words+['br']]
    stopped_tokens2 = [i for i in stopped_tokens if len(i)>1]
    text.append(stopped_tokens2)


In [7]:
text[:10]

[['students',
  'reality',
  'age',
  'far',
  'scramble',
  'years',
  'pathetic',
  'survive',
  'believe',
  'whole',
  'inspector',
  'think',
  'adults',
  'much',
  'high',
  'classic',
  'teachers',
  'closer',
  'one',
  'welcome',
  'many',
  'satire',
  'expect',
  'see',
  'sack',
  'episode',
  'burn',
  'pity',
  'right',
  'fetched',
  'saw',
  'life',
  'recalled',
  'ran',
  'repeatedly',
  'schools',
  'time',
  'pettiness',
  'pomp',
  'knew',
  'remind',
  'programs',
  '35',
  'student',
  'cartoon',
  'tried',
  'situation',
  'teaching',
  'lead',
  'school',
  'immediately',
  'insightful',
  'financially',
  'bromwell',
  'comedy',
  'line',
  'profession'],
 ['mutual',
  'saddles',
  'month',
  'survivors',
  'without',
  'hand',
  'dancer',
  'compared',
  'make',
  'characters',
  'prior',
  'years',
  'george',
  'rich',
  'warren',
  'meets',
  'matter',
  'stars',
  'tambor',
  'making',
  'racism',
  'wants',
  'got',
  'howard',
  'entertainment',
  'ex'

In [8]:
model = Word2Vec(text, vector_size=10, sg=1, window=2, min_count=3)
    # word2vec 모형 생성 , sg=1 skip-gram을 적용(중심단어로 주변단어 예측)
    # window=2 중심 단어로부터 좌우 2개의 단어, 왼쪽2, 오른쪽 2개까지 학습에 적용
    # min_count=3 최소 3회 이상 출현한 단어들을 대상으로 학습, 출현빈도 3회미만은 버림

# 두 단어의 유사도 계산
model.wv.similarity('film', 'movie')


0.94030154

In [9]:
# good과 가장 유사한 단어 5개
model.wv.most_similar("good",topn =5)
# model.wv.most_similar("hello",topn =5)   # 사전에 없는 단어는 에러가남: 많은 단어의 학습이 필요

[('like', 0.9009891152381897),
 ('take', 0.8781965374946594),
 ('actors', 0.8716089725494385),
 ('maybe', 0.8677181601524353),
 ('go', 0.8608437180519104)]

In [10]:
# 모델에 저장된 단어의 갯수

len(model.wv.index_to_key)

895

In [11]:
# 모델에 저장된 단어 텍스트
model.wv.index_to_key[0]

'movie'

In [12]:
# 단어에 해당하는 벡터값
model.wv.vectors[0]    # 단어하나가 10개의 숫자로 표현됨

array([ 0.14735173, -0.10042439,  0.2703968 ,  0.0300838 ,  0.05042636,
       -0.04672984,  0.2421013 ,  0.30999798, -0.270811  , -0.16221496],
      dtype=float32)